In [1]:
import os
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
import math
import json
import gc

In [2]:
listofReadFile = []
# reading the name of legal documents
for file in os.listdir("LegalCorpus"):
    if file.endswith(".txt"):
        listofReadFile.append(file)

In [3]:
dictionaryParagraphCorpus = {}
dictionaryParagraphWordCorpus = {}

countFile = 1

print('Building...')
for fileName in listofReadFile:
    #reading the document
    file = open('LegalCorpus/'+fileName, 'r', encoding="utf8")

    #reading the contents of each document as string
    listofParagraph = []
    listofParagraph = (file.read()).strip().split("\n")
    file.close()

    # formatting the sentence of a document
    temp = []
    for eachLine in listofParagraph[:]:
        eachLine = re.sub(r'(\d\d\d|\d\d|\d)\.\s', ' ', eachLine)#removes the paragraph lables 1. or 2. etc.
        eachLine = re.sub(r'(?<=[a-zA-Z])\.(?=\d)', '', eachLine)#removes dot(.) i.e File No.1063
        eachLine = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s[\da-z])', ' ', eachLine)#to remove the ending dot of abbr
        eachLine = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s?[\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~])', '', eachLine)#to remove the ending dot of abbr
        temp.append(eachLine.strip())
    listofParagraph = []
    listofParagraph = temp

    # Finding stop words
    stop_words = set(stopwords.words('english'))
    
    tempPara = []
    tempPara = listofParagraph.copy()
    listofParagraph = []
    paraGraphBagOfWord = []
    #Tokenising the contents of each document
    for eachParagraph in tempPara[:]:
        #Tokenising the document into sentences
        tokenisedSentences = []
        tokenisedSentences = nltk.sent_tokenize(eachParagraph)
        temp = []
        temp = tokenisedSentences.copy()
        bagOfWords = []
        lemmatizedWords = []

        line = ""
        for eachLine in temp[:]:
            line = eachLine
            eachLine = re.sub(r'[\.\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]', '', eachLine)
            tokenisedWords = nltk.word_tokenize(eachLine.lower())

            # removing the stopwords from a sentence
            withoutStopwords = [eachWord for eachWord in tokenisedWords if not eachWord in stop_words]

            # removing the punctuation from a sentence
            withoutStopwords = [eachWord for eachWord in withoutStopwords if not eachWord in string.punctuation]

            lemmatizer = nltk.WordNetLemmatizer()
            for words, tags in nltk.tag.pos_tag(withoutStopwords):
                if(tags.startswith('JJ')):
                    lemmatizedWords.append(lemmatizer.lemmatize(words, pos='a'))
                elif(tags.startswith('VB')):
                    lemmatizedWords.append(lemmatizer.lemmatize(words, pos='v'))
                elif(tags.startswith('RB')):
                    lemmatizedWords.append(lemmatizer.lemmatize(words, pos='r'))
                elif(tags.startswith('NN')):
                    lemmatizedWords.append(lemmatizer.lemmatize(words, pos='n'))
                else:
                    lemmatizedWords.append(lemmatizer.lemmatize(words))

            if(lemmatizedWords != []):
                bagOfWords.append(lemmatizedWords)
            else:
                del tokenisedSentences[tokenisedSentences.index(line)]
            lemmatizedWords = []
        
        #storing the paragraphs with sentence in each para : list[paragraph[sent]]
        listofParagraph.append(tokenisedSentences)
        #storing bag of words for paragraph : list[paragraph[tokenised words of sent]]
        paraGraphBagOfWord.append(bagOfWords)
            
    # Save words
    dictionaryParagraphWordCorpus[fileName] = paraGraphBagOfWord
    
    # Paragraph Corpus
    totalWordsinDocument = 0
    for eachPara in listofParagraph[:]:
        for eachSent in eachPara[:]:
            sent = eachSent.strip().split(' ')
            totalWordsinDocument += len(sent)
    totalWordsinDocument = math.ceil(totalWordsinDocument * 0.33)

    # Save sentences
    dictionaryParagraphCorpus[fileName] = [listofParagraph, totalWordsinDocument]
    
    # visuals
    print (countFile, '>', fileName, end = '  ')
    if(countFile % 10 == 0):
        print('\n')
    countFile += 1
    
    gc.collect()

Building...
1 > 1953_24.txt  2 > 1953_28.txt  3 > 1957_52.txt  4 > 1958_3.txt  5 > 1959_151.txt  6 > 1961_34.txt  7 > 1962_303.txt  8 > 1963_170.txt  9 > 1964_144.txt  10 > 1965_111.txt  

11 > 1966_154.txt  12 > 1966_236.txt  13 > 1967_267.txt  14 > 1968_197.txt  15 > 1969_324.txt  16 > 1969_57.txt  17 > 1971_111.txt  18 > 1971_138.txt  19 > 1971_336.txt  20 > 1971_49.txt  

21 > 1972_11.txt  22 > 1972_291.txt  23 > 1972_31.txt  24 > 1972_62.txt  25 > 1973_186.txt  26 > 1976_176.txt  27 > 1976_200.txt  28 > 1976_43.txt  29 > 1979_104.txt  30 > 1979_110.txt  

31 > 1979_158.txt  32 > 1979_33.txt  33 > 1980_221.txt  34 > 1980_39.txt  35 > 1980_91.txt  36 > 1981_49.txt  37 > 1982_28.txt  38 > 1983_129.txt  39 > 1983_27.txt  40 > 1983_37.txt  

41 > 1984_115.txt  42 > 1984_118.txt  43 > 1985_113.txt  44 > 1985_114.txt  45 > 1986_218.txt  46 > 1987_154.txt  47 > 1987_189.txt  48 > 1987_315.txt  49 > 1987_37.txt  50 > 1988_88.txt  

51 > 1989_233.txt  52 > 1990_171.txt  53 > 1990_96.txt  54

391 > 2010_J_30.txt  392 > 2010_J_31.txt  393 > 2010_J_34.txt  394 > 2010_J_38.txt  395 > 2010_J_41.txt  396 > 2010_J_43.txt  397 > 2010_J_44.txt  398 > 2010_J_45.txt  399 > 2010_J_48.txt  400 > 2010_J_5.txt  

401 > 2010_J_51.txt  402 > 2010_J_53.txt  403 > 2010_J_54.txt  404 > 2010_J_55.txt  405 > 2010_J_7.txt  406 > 2010_J_8.txt  407 > 2010_K_10.txt  408 > 2010_K_13.txt  409 > 2010_K_14.txt  410 > 2010_K_15.txt  

411 > 2010_K_16.txt  412 > 2010_K_18.txt  413 > 2010_K_2.txt  414 > 2010_K_20.txt  415 > 2010_K_22.txt  416 > 2010_K_24.txt  417 > 2010_K_25.txt  418 > 2010_K_26.txt  419 > 2010_K_28.txt  420 > 2010_K_30.txt  

421 > 2010_K_32.txt  422 > 2010_K_33.txt  423 > 2010_K_34.txt  424 > 2010_K_41.txt  425 > 2010_K_42.txt  426 > 2010_K_43.txt  427 > 2010_K_47.txt  428 > 2010_K_48.txt  429 > 2010_K_5.txt  430 > 2010_K_51.txt  

431 > 2010_K_52.txt  432 > 2010_K_53.txt  433 > 2010_K_55.txt  434 > 2010_K_58.txt  435 > 2010_K_6.txt  436 > 2010_K_61.txt  437 > 2010_K_62.txt  438 > 2010_

771 > 2010_S_190.txt  772 > 2010_S_193.txt  773 > 2010_S_194.txt  774 > 2010_S_195.txt  775 > 2010_S_196.txt  776 > 2010_S_199.txt  777 > 2010_S_20.txt  778 > 2010_S_201.txt  779 > 2010_S_202.txt  780 > 2010_S_203.txt  

781 > 2010_S_21.txt  782 > 2010_S_210.txt  783 > 2010_S_212.txt  784 > 2010_S_213.txt  785 > 2010_S_216.txt  786 > 2010_S_217.txt  787 > 2010_S_218.txt  788 > 2010_S_219.txt  789 > 2010_S_222.txt  790 > 2010_S_223.txt  

791 > 2010_S_224.txt  792 > 2010_S_225.txt  793 > 2010_S_226.txt  794 > 2010_S_227.txt  795 > 2010_S_23.txt  796 > 2010_S_231.txt  797 > 2010_S_232.txt  798 > 2010_S_233.txt  799 > 2010_S_234.txt  800 > 2010_S_236.txt  

801 > 2010_S_237.txt  802 > 2010_S_238.txt  803 > 2010_S_241.txt  804 > 2010_S_242.txt  805 > 2010_S_243.txt  806 > 2010_S_245.txt  807 > 2010_S_246.txt  808 > 2010_S_248.txt  809 > 2010_S_249.txt  810 > 2010_S_250.txt  

811 > 2010_S_251.txt  812 > 2010_S_252.txt  813 > 2010_S_255.txt  814 > 2010_S_26.txt  815 > 2010_S_260.txt  816 > 

1131 > 2010_U_67.txt  1132 > 2010_U_68.txt  1133 > 2010_U_7.txt  1134 > 2010_U_71.txt  1135 > 2010_U_73.txt  1136 > 2010_U_74.txt  1137 > 2010_U_77.txt  1138 > 2010_U_79.txt  1139 > 2010_U_8.txt  1140 > 2010_U_82.txt  

1141 > 2010_U_83.txt  1142 > 2010_U_84.txt  1143 > 2010_U_86.txt  1144 > 2010_U_90.txt  1145 > 2010_U_94.txt  1146 > 2010_U_95.txt  1147 > 2010_U_96.txt  1148 > 2010_U_97.txt  1149 > 2010_U_98.txt  1150 > 2010_V_1.txt  

1151 > 2010_V_10.txt  1152 > 2010_V_12.txt  1153 > 2010_V_15.txt  1154 > 2010_V_17.txt  1155 > 2010_V_18.txt  1156 > 2010_V_20.txt  1157 > 2010_V_22.txt  1158 > 2010_V_27.txt  1159 > 2010_V_28.txt  1160 > 2010_V_31.txt  

1161 > 2010_V_32.txt  1162 > 2010_V_35.txt  1163 > 2010_V_38.txt  1164 > 2010_V_40.txt  1165 > 2010_V_44.txt  1166 > 2010_V_45.txt  1167 > 2010_V_47.txt  1168 > 2010_V_48.txt  1169 > 2010_V_6.txt  1170 > 2010_V_7.txt  

1171 > 2010_V_8.txt  1172 > 2010_W_1.txt  1173 > 2010_W_4.txt  1174 > 2010_W_5.txt  1175 > 2010_W_6.txt  1176 > 2010_

1501 > 2011_M_75.txt  1502 > 2011_M_77.txt  1503 > 2011_M_79.txt  1504 > 2011_M_8.txt  1505 > 2011_M_80.txt  1506 > 2011_M_82.txt  1507 > 2011_M_83.txt  1508 > 2011_M_84.txt  1509 > 2011_M_85.txt  1510 > 2011_M_86.txt  

1511 > 2011_M_88.txt  1512 > 2011_M_9.txt  1513 > 2011_M_90.txt  1514 > 2011_M_92.txt  1515 > 2011_M_93.txt  1516 > 2011_M_94.txt  1517 > 2011_M_95.txt  1518 > 2011_M_96.txt  1519 > 2011_M_97.txt  1520 > 2011_N_1.txt  

1521 > 2011_N_10.txt  1522 > 2011_N_13.txt  1523 > 2011_N_14.txt  1524 > 2011_N_16.txt  1525 > 2011_N_17.txt  1526 > 2011_N_19.txt  1527 > 2011_N_20.txt  1528 > 2011_N_21.txt  1529 > 2011_N_25.txt  1530 > 2011_N_28.txt  

1531 > 2011_N_29.txt  1532 > 2011_N_3.txt  1533 > 2011_N_31.txt  1534 > 2011_N_36.txt  1535 > 2011_N_37.txt  1536 > 2011_N_38.txt  1537 > 2011_N_39.txt  1538 > 2011_N_4.txt  1539 > 2011_N_42.txt  1540 > 2011_N_43.txt  

1541 > 2011_N_44.txt  1542 > 2011_N_45.txt  1543 > 2011_N_47.txt  1544 > 2011_N_49.txt  1545 > 2011_N_5.txt  1546 > 2

...

5861 > 2016_M_44.txt  5862 > 2016_M_46.txt  5863 > 2016_M_47.txt  5864 > 2016_M_5.txt  5865 > 2016_M_50.txt  5866 > 2016_M_53.txt  5867 > 2016_M_54.txt  5868 > 2016_M_55.txt  5869 > 2016_M_58.txt  5870 > 2016_M_59.txt  

5871 > 2016_M_6.txt  5872 > 2016_M_61.txt  5873 > 2016_M_62.txt  5874 > 2016_M_63.txt  5875 > 2016_M_65.txt  5876 > 2016_M_67.txt  5877 > 2016_M_7.txt  5878 > 2016_M_73.txt  5879 > 2016_M_76.txt  5880 > 2016_M_77.txt  

5881 > 2016_M_82.txt  5882 > 2016_M_85.txt  5883 > 2016_M_86.txt  5884 > 2016_M_89.txt  5885 > 2016_M_91.txt  5886 > 2016_M_94.txt  5887 > 2016_M_95.txt  5888 > 2016_M_98.txt  5889 > 2016_M_99.txt  5890 > 2016_N_1.txt  

5891 > 2016_N_10.txt  5892 > 2016_N_13.txt  5893 > 2016_N_17.txt  5894 > 2016_N_2.txt  5895 > 2016_N_20.txt  5896 > 2016_N_31.txt  5897 > 2016_N_33.txt  5898 > 2016_N_37.txt  5899 > 2016_N_41.txt  5900 > 2016_N_42.txt  

5901 > 2016_N_44.txt  5902 > 2016_N_48.txt  5903 > 2016_N_49.txt  5904 > 2016_N_5.txt  5905 > 2016_N_50.txt  5906 > 2

6221 > 2016_S_374.txt  6222 > 2016_S_376.txt  6223 > 2016_S_377.txt  6224 > 2016_S_379.txt  6225 > 2016_S_380.txt  6226 > 2016_S_381.txt  6227 > 2016_S_382.txt  6228 > 2016_S_386.txt  6229 > 2016_S_388.txt  6230 > 2016_S_389.txt  

6231 > 2016_S_39.txt  6232 > 2016_S_392.txt  6233 > 2016_S_394.txt  6234 > 2016_S_396.txt  6235 > 2016_S_398.txt  6236 > 2016_S_400.txt  6237 > 2016_S_402.txt  6238 > 2016_S_403.txt  6239 > 2016_S_405.txt  6240 > 2016_S_406.txt  

6241 > 2016_S_407.txt  6242 > 2016_S_411.txt  6243 > 2016_S_412.txt  6244 > 2016_S_417.txt  6245 > 2016_S_418.txt  6246 > 2016_S_42.txt  6247 > 2016_S_421.txt  6248 > 2016_S_423.txt  6249 > 2016_S_425.txt  6250 > 2016_S_426.txt  

6251 > 2016_S_428.txt  6252 > 2016_S_429.txt  6253 > 2016_S_43.txt  6254 > 2016_S_435.txt  6255 > 2016_S_438.txt  6256 > 2016_S_439.txt  6257 > 2016_S_44.txt  6258 > 2016_S_441.txt  6259 > 2016_S_443.txt  6260 > 2016_S_448.txt  

6261 > 2016_S_449.txt  6262 > 2016_S_450.txt  6263 > 2016_S_451.txt  6264 > 

6581 > 2017_C_62.txt  6582 > 2017_C_63.txt  6583 > 2017_C_66.txt  6584 > 2017_C_67.txt  6585 > 2017_C_68.txt  6586 > 2017_C_70.txt  6587 > 2017_C_71.txt  6588 > 2017_C_72.txt  6589 > 2017_C_74.txt  6590 > 2017_C_76.txt  

6591 > 2017_C_77.txt  6592 > 2017_C_79.txt  6593 > 2017_C_80.txt  6594 > 2017_C_82.txt  6595 > 2017_C_87.txt  6596 > 2017_C_88.txt  6597 > 2017_C_90.txt  6598 > 2017_C_99.txt  6599 > 2017_D_10.txt  6600 > 2017_D_100.txt  

6601 > 2017_D_105.txt  6602 > 2017_D_108.txt  6603 > 2017_D_109.txt  6604 > 2017_D_111.txt  6605 > 2017_D_112.txt  6606 > 2017_D_117.txt  6607 > 2017_D_12.txt  6608 > 2017_D_122.txt  6609 > 2017_D_123.txt  6610 > 2017_D_127.txt  

6611 > 2017_D_13.txt  6612 > 2017_D_130.txt  6613 > 2017_D_20.txt  6614 > 2017_D_24.txt  6615 > 2017_D_26.txt  6616 > 2017_D_3.txt  6617 > 2017_D_30.txt  6618 > 2017_D_32.txt  6619 > 2017_D_36.txt  6620 > 2017_D_4.txt  

6621 > 2017_D_42.txt  6622 > 2017_D_43.txt  6623 > 2017_D_45.txt  6624 > 2017_D_46.txt  6625 > 2017_D_4

6951 > 2017_P_57.txt  6952 > 2017_P_60.txt  6953 > 2017_P_61.txt  6954 > 2017_P_62.txt  6955 > 2017_P_7.txt  6956 > 2017_P_76.txt  6957 > 2017_P_77.txt  6958 > 2017_P_80.txt  6959 > 2017_P_84.txt  6960 > 2017_P_85.txt  

6961 > 2017_P_91.txt  6962 > 2017_P_94.txt  6963 > 2017_P_95.txt  6964 > 2017_P_97.txt  6965 > 2017_R_10.txt  6966 > 2017_R_102.txt  6967 > 2017_R_103.txt  6968 > 2017_R_105.txt  6969 > 2017_R_112.txt  6970 > 2017_R_116.txt  

6971 > 2017_R_12.txt  6972 > 2017_R_120.txt  6973 > 2017_R_126.txt  6974 > 2017_R_127.txt  6975 > 2017_R_13.txt  6976 > 2017_R_130.txt  6977 > 2017_R_135.txt  6978 > 2017_R_14.txt  6979 > 2017_R_140.txt  6980 > 2017_R_141.txt  

6981 > 2017_R_142.txt  6982 > 2017_R_143.txt  6983 > 2017_R_149.txt  6984 > 2017_R_16.txt  6985 > 2017_R_2.txt  6986 > 2017_R_24.txt  6987 > 2017_R_25.txt  6988 > 2017_R_31.txt  6989 > 2017_R_32.txt  6990 > 2017_R_36.txt  

6991 > 2017_R_37.txt  6992 > 2017_R_39.txt  6993 > 2017_R_41.txt  6994 > 2017_R_44.txt  6995 > 2017

7301 > 2017_U_15.txt  7302 > 2017_U_151.txt  7303 > 2017_U_153.txt  7304 > 2017_U_155.txt  7305 > 2017_U_158.txt  7306 > 2017_U_16.txt  7307 > 2017_U_163.txt  7308 > 2017_U_165.txt  7309 > 2017_U_167.txt  7310 > 2017_U_169.txt  

7311 > 2017_U_170.txt  7312 > 2017_U_172.txt  7313 > 2017_U_173.txt  7314 > 2017_U_178.txt  7315 > 2017_U_183.txt  7316 > 2017_U_19.txt  7317 > 2017_U_22.txt  7318 > 2017_U_28.txt  7319 > 2017_U_31.txt  7320 > 2017_U_34.txt  

7321 > 2017_U_35.txt  7322 > 2017_U_39.txt  7323 > 2017_U_4.txt  7324 > 2017_U_41.txt  7325 > 2017_U_42.txt  7326 > 2017_U_46.txt  7327 > 2017_U_47.txt  7328 > 2017_U_55.txt  7329 > 2017_U_59.txt  7330 > 2017_U_62.txt  

7331 > 2017_U_66.txt  7332 > 2017_U_70.txt  7333 > 2017_U_71.txt  7334 > 2017_U_73.txt  7335 > 2017_U_75.txt  7336 > 2017_U_77.txt  7337 > 2017_U_78.txt  7338 > 2017_U_8.txt  7339 > 2017_U_82.txt  7340 > 2017_U_86.txt  

7341 > 2017_U_91.txt  7342 > 2017_U_93.txt  7343 > 2017_U_95.txt  7344 > 2017_U_97.txt  7345 > 2017_V

In [4]:
#saving dictionary of paragraph to .json file
with open('ParagraphCorpus.json', 'w') as fp:
    json.dump(dictionaryParagraphCorpus, fp, sort_keys=True, indent=0)
fp.close()
gc.collect()

33

In [5]:
#saving dictionary of paragraph to .json file
with open('ParagraphWordCorpus.json', 'w') as fp:
    json.dump(dictionaryParagraphWordCorpus, fp, sort_keys=True, indent=0)
fp.close()
gc.collect()

33